In [1]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from implicit.evaluation import train_test_split
from cv_py import CrossValidation
from scipy import sparse

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

[Errno 2] No such file or directory: 'C:Users781110104OneDrive - GenpactDocumentsVSCode'
/home/cbarkey


In [53]:
dl = DataLoader()

In [3]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

In [6]:
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [17]:
# initial split 80/20
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [5]:
cv = CrossValidation(5)

In [19]:
# perform k-fold splitting of trining data
validate, calibrate = cv.split_k_fold(train, 22)
validate_t, calibrate_t = cv.split_k_fold(train_t, 22)

Tune models for OEM1 data:

In [22]:
space_iALS = {'factors' : [64], 'regularization' : [80, 90, 100, 110, 120, 130, 140, 150, 160, 170], 'alpha' : [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], 'iterations' : [15]}
hyper_ials = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')

In [ ]:
space_eALS = {'factors' : [64], 'regularization' : [220, 230, 240, 250, 260, 270], 'alpha' : [0.05, 0.1, 0.15, 0.2], 'w0' : [50000, 55000, 60000, 65000], 'iterations' : [15]}
hyper_eals = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_eALS, model_class='eALS')

In [ ]:
space_BPR = {'factors' : [64], 'regularization' : [0.005, 0.01, 0.02, 0.03, 0.04], 'learning_rate' : [0.003, 0.005, 0.007, 0.009, 0.01, 0.02, 0.03], 'iterations' : [15]}
hyper_bpr = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_BPR, model_class='BPR')

In [ ]:
space_LMF = {'factors' : [64], 'regularization' : [5, 10, 20, 30, 40, 50], 'learning_rate' : [0.8, 1.0, 1.2, 1.4], 'iterations' : [15], 'neg_prop': [40, 50, 60, 70, 80]}
hyper_lmf = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_LMF, model_class='LMF')

Load FM data for OEM1:

In [ ]:
interactions_csr = sparse.load_npz('interactions_csr.npz')
weights_csr = sparse.load_npz('weights_csr.npz')
itemf = sparse.load_npz('itemf.npz')
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)
val_w, cal_w = cv.split_k_fold(train_w, 22)
val_i, cal_i = cv.split_k_fold(train_i, 22)

In [ ]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0001, 0.0005, 0.001], 'learning_rate' : [0.01, 0.05, 0.1], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [90, 100, 110]}
hyper_fm = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp, model_class='FM', item_features=itemf)

Tune models for OEM2 data:

In [23]:
space_iALS_t = {'factors' : [64], 'regularization' : [50, 60, 70, 80, 90, 100, 110], 'alpha' : [0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5], 'iterations' : [15]}
hyper_ials_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')

In [ ]:
space_eALS_t = {'factors' : [64], 'regularization' : [5, 10, 15], 'alpha' : [0.05, 0.1, 0.2, 0.3], 'w0' : [1000, 1500, 2000, 2500], 'iterations' : [15]}
hyper_eals_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_eALS_t, model_class='eALS')

In [ ]:
space_BPR_t = {'factors' : [64], 'regularization' : [0.03, 0.035, 0.04, 0.045], 'learning_rate' : [0.025, 0.03, 0.035, 0.04], 'iterations' : [15]}
hyper_bpr_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_BPR_t, model_class='BPR')

In [ ]:
space_LMF_t = {'factors' : [64], 'regularization' : [5, 10, 20, 30], 'learning_rate' : [0.8, 1.0, 1.2, 1.4], 'iterations' : [15], 'neg_prop': [0, 1, 2, 3, 5, 10]}
hyper_lmf_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_LMF_t, model_class='LMF')

Load FM data for OEM2:

In [45]:
interactions_csr_t = sparse.load_npz('interactions_csr_t.npz')
weights_csr_t = sparse.load_npz('weights_csr_t.npz')
userf_t = sparse.load_npz('userf_t.npz')
train_i_t, test_i_t = train_test_split(interactions_csr, 0.8, 22)
train_w_t, test_w_t = train_test_split(weights_csr, 0.8, 22)
val_w_t, cal_w_t = cv.split_k_fold(train_w, 22)
val_i_t, cal_i_t = cv.split_k_fold(train_i, 22)

In [56]:
space_FM_warp_t = {'factors' : [64], 'regularization' : [0.00005, 0.0001, 0.0005], 'learning_rate' : [0.0005, 0.001, 0.005], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [4, 5, 6]}
hyper_fm_t = cv.hyperp_tuning(test=val_w_t, train=cal_w_t, exclude=test_w_t, seed=22, param_space=space_FM_warp_t, model_class='FM', user_features=userf_t)